# Machine Learning

### What is Machine Learning?

Machine learning explores the study and construction of algorithms that can learn from and make predictions on **data**. Data allows us to evaluate how good or bad the algorithm is doing. Based on the evaluation, we can update the algorithm to perform "better".

There are two main categories of machine learning algorithms: supervised and unsupervised learning. 

Supervised learning is the task of constructing an algorithm to perform predictions with labeled data. For example, if we are trying to predict windmill power generated we may want to gather data on temperature, wind speed, time of day (features) and the power generated (labels).

Unsupervised learning learning is the task of constructing an algorithm without labeled data.

### Examples
- Windmill power forecasting
- Stock price prediction
- Predicting capacitor failure
- Image classification
- News article clustering

### Data

The data used will likely be able to be structured in a csv (excel spreadsheet). Data is usually represented as one row per observation. Each column represents a feature of that observation.

Example: Dataset from kaggle 

<img src="images/csv_image.png" />
[Source](http://blog.trifork.com/2017/02/16/machine-learning-predicting-house-prices/)

In [ ]:
import sklearn.datasets as datasets
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib nbagg
%matplotlib inline

In [ ]:
np.random.seed(0)

In [ ]:
n_features = 1   # Will not plot if this variable is changed
noise = 10.
# bias = np.random.rand() * 100

In [ ]:
X, y = datasets.make_regression(n_features=n_features, noise=noise) # bias=bias)
N = X.shape[0]

In [ ]:
plt.scatter(X, y)

# Linear Regression

We can choose our algorithm to be a linear model. Our algorithm will have the form:

$$\hat{y} = WX + b$$

where $\hat{y}$ is our prediction for $y$, $X$ is the features, $W$ is the weights for each feature, and $b$ is the intercept. Linear regression can be used to model data with labels that are continuous.

*Note: uppercase variables are matrices and lowercase variables are vectors.*

### How do we find the right W?

We have already chosen our algorithm, now we have to define how well that algorithm does. This will be called our cost function:

$$ C = \frac{1}{2N}\sum_{i=1}^{N}(y_i - \hat{y}_i)^2 $$

where $y$ is the true value, $\hat{y}$ is our prediction for $y$, and $N$ is the number of observations. $C$ is our cost function which will evaluate how good our model is (the lower the better).

Since this example uses a single variable we can easily visualize the cost for all $W$.

In [ ]:
def f(X, W, b=None):
    if b:
        y_hat = X.dot(W.T) + b
    else:
        y_hat = X.dot(W.T)
    return y_hat.reshape(-1)

def cost(X, W, y, b=None):
    N = X.shape[0]
    y_hat = f(X, W, b)
    C = (1./ (2. * N)) * np.sum((y - y_hat) ** 2)
    return C

In [ ]:
Ws = np.linspace(30, 55)
Cs = []
for w in Ws:
    w = np.array(w)
    Cs.append(cost(X, w, y))
plt.scatter(Ws, Cs)

### Gradient Descent

The set of $W$ and $b$s that we want is the one that gives us the lowest cost. With some calculus we can calculate the derivate of $C$ with respect to $W$. Using the derivative, we can find the optimal $W$. (Remember the chain rule).

$$ \frac{dC}{d\hat{y}} = \frac{1}{N}(y - \hat{y}) $$
$$ \frac{d\hat{y}}{dW} = X $$
$$ \frac{dC}{dW} = \frac{dC}{d\hat{y}}\frac{d\hat{y}}{dW} = (y - \hat{y}) \cdot X $$

We can do a similar calculation for $b$, except $\frac{d\hat{y}}{db} = 1$.

We will use the direction of the derivative to find the lowest point of the function, which represents the minimum of the cost.

In [ ]:
def grad(X, W, y, b=None):
    N = X.shape[0]
    y_hat = f(X, W, b)
    diff = (y_hat - y)
    gradW = diff.dot(X) / N
    gradb = np.sum(diff) / N
    return gradW, gradb

We can start with the initial choice for $W$ and $b$ to be all zeros. We will then use gradient descent to find the optimal $W$ and $b$.

In [ ]:
learning_rate = 0.25

W = np.zeros((1, n_features))
b = 0

Xs = np.linspace(-3, 3)    # for plotting
costs = []

In [ ]:
# Execute this cell to iterate and visualize the updated weights

gradW, gradb = grad(X, W, y, b)
W -= learning_rate * gradW
b -= learning_rate * gradb
costs.append(cost(X, W, y, b))

y_hat = f(Xs.reshape(-1, 1), W, b)
plt.scatter(X, y)
plt.plot(Xs, y_hat)

We can also visualize how the cost goes down for every iteration

In [ ]:
plt.plot(range(len(costs)), costs)

# Logistic Regression

Logistic regression is used for classification problems. Typically, the labels are binary and can be represented as 1/0 (pass/fail). We will need a new algorithm to model our data. We will use the sigmoid function. This function squashes the output to between 0 and 1, like a probability.

$$ \sigma (z) = \hat{y} = \frac{1}{1 + e^{-z}} $$

We will refer to the sigmoid function as $\sigma (z)$, where $z = WX + b$.

The function looks like this.

<img src="images/sigmoid.png" />

We will also need to modify our cost function. We will want to heavily penalize predictions close to 0 when the true value is close to 1 and vice-versa. To do this we can use the log-loss function.

$$ C = -\frac{1}{N} \sum_{i=1}^{N} y_i log(\hat{y}_i) + (1 - y_i) log(1 - \hat{y}_i) $$

The first term in the summation is multiplied by the true value, $y$. Notice when this is 0, this term will also be 0. Also, the second term will be 0, when the $y$ is 1. This leaves us with the log($\hat{y}$) which is large when $\hat{y}$ is close to 0 and small when $\hat{y}$ is close to 1. This cost functions characterizes how we will penalize strong predictions of the opposite class.

In [ ]:
eps = 1e-7    # use so we don't get log(0)

def sigmoid(z):
    y_hat = 1. / (1 + np.exp(-z))
    return y_hat

def log_loss(X, W, y, b=None):
    N = X.shape[0]
    z = f(X, W, b)
    y_hat = sigmoid(z)
    y_hat = np.clip(y_hat, eps, 1 - eps)
    C = -(1./N) * np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    return C

This time let's use 2 features since we can use a color to visualize the label.

In [ ]:
np.random.seed(10)

In [ ]:
n_features = 2

In [ ]:
def get_colors(y_):
    return ['b' if p == 1 else 'r' for p in y_]

In [ ]:
X, y = datasets.make_classification(n_features=n_features, n_informative=n_features, n_redundant=0, shift=5.)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], color=get_colors(y))

### Gradient Descent (part 2)

Again, we will use gradient descent to calculate the optimal $W$ and $b$. The gradient has also changed since we updated our cost function. Doing some calculus we get the derivate as:

$$ \frac{dlog(C)}{d\hat{y}} = \hat{y} - y $$
$$ \frac{d\hat{y}}{dW} = -\sigma (z) (1 - \sigma (z)) \cdot X $$

Refer to [here](https://en.wikipedia.org/wiki/Logistic_function) for more math.

In [ ]:
def grad_log(X, W, y, b=None):
    N = X.shape[0]
    z = f(W, X, b)
    y_hat = sigmoid(z)
    gradW = y_hat - y
    gradW *= y_hat * (1 - y_hat)
    gradW = gradW.reshape(1, -1).dot(X)
    gradb = y_hat - y
    gradb *= y_hat * (1 - y_hat)
    gradb = np.sum(gradb)
    return gradW, gradb

In [ ]:
learning_rate = 0.005

W = np.zeros((1, n_features))
b = 0

costs = []

In [ ]:
gradW, gradb = grad_log(X, W, y, b)
W -= learning_rate * gradW
b -= learning_rate * gradb
costs.append(log_loss(X, W, y, b))

z = f(X, W, b)
y_hat = sigmoid(z)
y_hat = y_hat > 0.5    # Make binary predictions (0/1)

plt.scatter(X[:,0], X[:,1], color=get_colors(y_hat))

Visualize the cost for each iteration

In [ ]:
plt.plot(range(len(costs)), costs)